In [13]:
# imports
import torch
import evaluate
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead, create_reference_model

from src.constants import MT_MODEL, MT_DATA_FILE, MT_SEED
from src.metrics import translation_reward

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

0

In [15]:
translations = pd.read_csv(MT_DATA_FILE)
train_dataset, test_dataset = train_test_split(translations, test_size=0.8, random_state=MT_SEED)

In [16]:
# get models
model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(MT_MODEL).to(device)
model_ref = create_reference_model(model).to(device)

tokenizer = AutoTokenizer.from_pretrained(MT_MODEL)

# initialize trainer
ppo_config = PPOConfig(
    batch_size=1,
    learning_rate=1.41e-5,
    #log_with="wandb",
)

# create a ppo trainer
ppo_trainer = PPOTrainer(ppo_config, model, model_ref, tokenizer)

# reward model
bleu = evaluate.load("bleu")

c:\Users\aleks\Documents\Moje dokumenty\Studia\Semestr 9\USD\USD-trl\.venv\lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
c:\Users\aleks\Documents\Moje dokumenty\Studia\Semestr 9\USD\USD-trl\.venv\lib\site-packages\trl\trainer\ppo_trainer.py:257: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [17]:
for epoch in range(1):
    for text, translation in zip(train_dataset['Polish'], train_dataset['English']):
        print("\n----------------------------")
        print(f'Source sentence: {text}')
        print(f'Target sentence: {translation}')
        # encode a query
        query_tensor = tokenizer.encode(text, return_tensors="pt").to(device)

        # get model response
        response_tensor = model.generate(input_ids=query_tensor)
        result_txt = [tokenizer.decode(response_tensor[0], skip_special_tokens=True)]
        print(f'Response sentence: {result_txt[0]}')

        # define a reward for response
        reward = translation_reward(result_txt, translation, bleu, device)
        print(f'Reward: {reward[0].item()}')

        # train model for one step with ppo
        train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)
        ppo_trainer.log_stats(train_stats, {text:result_txt}, reward)


----------------------------
Source sentence: Wszystko co wiecie o historii to kłamstwo. Na przykład:
Target sentence: Everything you think you know about history is a lie.
Response sentence: All you know about history is a lie.
Reward: 0.6956543326377869


c:\Users\aleks\Documents\Moje dokumenty\Studia\Semestr 9\USD\USD-trl\.venv\lib\site-packages\trl\trainer\ppo_trainer.py:1334: UserWarning: The game logs will not be logged because the batch does not contain the keys 'query' and 'response'. 
  warnings.warn(



----------------------------
Source sentence: Pewnie słyszał pan o sprawie Gabby Stone? Pan z Departamentu Sprawiedliwości?
Target sentence: I'm sure you've heard about the investigation regarding Gabby Stone.
Response sentence: You've probably heard of the Gabby Stone case?
Reward: 0.10000000149011612

----------------------------
Source sentence: Joey jest bardzo lojalny, ale nie masz pojęcia, ilu tam jest teraz wyznawców.
Target sentence: Joey is very loyal, but you have no idea how many followers are out there now.
Response sentence: Joey is very loyal, but you have no idea how many followers there are now.
Reward: 0.8495285511016846

----------------------------
Source sentence: Następne płatności dokonywane są zgodnie z zasadami przewidzianymi w art. 10.
Target sentence: Subsequent payments shall be made in accordance with the rules provided for in Article 10.
Response sentence: The following payments shall be made in accordance with the rules provided for in Article 10.
Reward:

c:\Users\aleks\Documents\Moje dokumenty\Studia\Semestr 9\USD\USD-trl\.venv\lib\site-packages\trl\trainer\ppo_trainer.py:1271: UserWarning: KL divergence is starting to become negative: -1.57 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(



----------------------------
Source sentence: w języku węgierskim: Fizikai ellenőrzés elvégezve [2535/2001/EK rendelet],
Target sentence: in Hungarian: Fizikai ellenőrzés elvégezve (2535/2001/EK rendelet),
Response sentence: in Hungarian: Fizikai ellenőrzés elvégezve [2535/2001/EK rendelet],
Reward: 0.6080942153930664

----------------------------
Source sentence: Możemy dziękować Bogu, że nie było większej tragedii.
Target sentence: And we can just thank God the tragedy wasn't worse.
Response sentence: We can thank God that there was no greater tragedy.
Reward: 0.0882352963089943

----------------------------
Source sentence: Bo jesteś kobietą moich marzeń i zasługujesz na idealne oświadczyny.
Target sentence: 'cause you're the woman of my dreams And you deserve the perfect proposal.
Response sentence: Because you're a woman of my dreams and you deserve a perfect proposal.
Reward: 0.4471047818660736


...

c:\Users\aleks\Documents\Moje dokumenty\Studia\Semestr 9\USD\USD-trl\.venv\lib\site-packages\trl\trainer\ppo_trainer.py:1271: UserWarning: KL divergence is starting to become negative: -1.60 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(



----------------------------
Source sentence: Dopiero zaczynamy odkrywać prawdziwą naturę inteligencji.
Target sentence: You see, the fact is that we're only now just beginning to understand the true nature of intelligence.
Response sentence: We're just beginning to discover the true nature of intelligence.
Reward: 0.3401889503002167


c:\Users\aleks\Documents\Moje dokumenty\Studia\Semestr 9\USD\USD-trl\.venv\lib\site-packages\trl\trainer\ppo_trainer.py:1271: UserWarning: KL divergence is starting to become negative: -2.51 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(



----------------------------
Source sentence: Podpisał wczoraj umowę z Fundacją Wiecznego Postępu.
Target sentence: He signed a contract with the Forever Forward Foundation yesterday.
Response sentence: He signed an agreement with the Foundation of Eternal Progress yesterday.
Reward: 0.20000000298023224


c:\Users\aleks\Documents\Moje dokumenty\Studia\Semestr 9\USD\USD-trl\.venv\lib\site-packages\trl\trainer\ppo_trainer.py:1271: UserWarning: KL divergence is starting to become negative: -1.00 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
